In [3]:
import numpy as np
from matplotlib import pyplot
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use specified GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as err:
        print(err)

tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [19]:
data_dir = "...\\celeb_dataset"
IMG_H = 64
IMG_W = 64
latent_dim = 128

## Model

In [20]:
w_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

In [21]:
def conv_transpose_block(inputs, num_filters, kernel_size=3, strides=2):
    x = Conv2DTranspose(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding="same",
        strides=strides,
        use_bias=False
    )(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [22]:
def conv_block(inputs, num_filters, kernel_size=3, padding="same", strides=2, activation=True):
    x = Conv2D(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding=padding,
        strides=strides,
    )(inputs)
    
    if activation:
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = GaussianNoise(0.2)(x)
    return x

In [23]:
def build_generator(latent_dim):
    filters = [1024,512,256,128,64,32]

    noise = Input(shape=(latent_dim,), name="latent_vector")

    x = Dense(filters[0] * 4 * 4)(noise)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Reshape((4, 4, filters[0]))(x)
    for i in range(1, 5):
        x = conv_transpose_block(x,num_filters=filters[i])
        
    x = conv_block(x,
        num_filters=3,
        strides=1,
        activation=False
    )
    fake_output = Activation("tanh")(x)
    return Model(noise, fake_output, name="generator")

In [24]:
#build_generator(latent_dim).summary()

In [25]:
def build_discriminator():
    filters = [128, 256, 512,1024]
    
    image_input = Input(shape=(IMG_H, IMG_W, 3))
    x = image_input

    for i in range(0, 4):
        x = conv_block(x, num_filters=filters[i])
    x = Flatten()(x)
    x = Dense(1)(x)
    return Model(image_input, x)

In [26]:
#build_discriminator().summary()

In [27]:
class DCGAN(Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(DCGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
    def compile(self, d_optimizer, g_optimizer, loss_function):
        super(DCGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_function = loss_function
    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        
        for _ in range(2):
            
            ## Discriminator step - fake images
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim)) 
            generated_images = self.generator(random_latent_vectors)
            generated_labels = tf.zeros((batch_size, 1))
            with tf.GradientTape() as ftape:
                predictions = self.discriminator(generated_images)
                df_loss = self.loss_function(generated_labels, predictions)
            grads = ftape.gradient(df_loss, self.discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

            ## Generator step
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            misleading_labels = tf.ones((batch_size, 1))
            with tf.GradientTape() as gtape:
                predictions = self.discriminator(self.generator(random_latent_vectors))
                g_loss = self.loss_function(misleading_labels, predictions)
            grads = gtape.gradient(g_loss, self.generator.trainable_weights)
            self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))
            
            ## Discriminator step - real images
            labels = tf.ones((batch_size, 1))
            with tf.GradientTape() as rtape:
                predictions = self.discriminator(real_images)
                dr_loss = self.loss_function(labels, predictions)
            grads = rtape.gradient(dr_loss, self.discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))
            
            ## Generator step
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            misleading_labels = tf.ones((batch_size, 1))
            with tf.GradientTape() as gtape:
                predictions = self.discriminator(self.generator(random_latent_vectors))
                g_loss = self.loss_function(misleading_labels, predictions)
            grads = gtape.gradient(g_loss, self.generator.trainable_weights)
            self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))
            
            return {"df_loss": df_loss, "dr_loss": dr_loss, "g_loss": g_loss}

## Preparation

In [28]:
# Create a data loader to not overwhelm the ram

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  shuffle = True,
  image_size=(IMG_W, IMG_H),
  batch_size=32)

Found 202599 files belonging to 1 classes.


In [29]:
# Dataset normalization

def preprocessing_image(img):
    img = (img - 127.5) / 127.5
    return img

normalized_ds = train_ds.map(lambda x, y: preprocessing_image(x))

In [30]:
n_samples = 16
noise = np.random.normal(size=(n_samples, latent_dim))

In [31]:
def save_plot(examples, index, n):
    examples = (examples + 1) / 2.0 # normalize from (-1, 1) range to (0, 1) range
    fig = pyplot.figure(figsize=(32, 32))
    #n = 4
    for i in range(n * n):
        fig.add_subplot(n, n, i+1)
        pyplot.axis("off")
        image = examples[i]
        pyplot.imshow(image)
    filename = f"samples/generated_plot_index-{index}.png"
    pyplot.savefig(filename)
    pyplot.close()

In [32]:
training_history = []
global index
index = 0

In [33]:
class custom_callback(keras.callbacks.Callback):

    def on_batch_end(self, batch, logs=None):
        global index
        training_history.append(logs)
        
        if (batch % 500 == 499):
            index += 1
            examples = g_model.predict(noise)
            save_plot(examples, index, int(np.sqrt(n_samples)))
            g_model.save(f"checkpoints/g_model_{str(index)}.h5")
            d_model.save(f"checkpoints/d_model_{str(index)}.h5")
            

## Train

In [34]:
d_model = build_discriminator()
g_model = build_generator(latent_dim)
#d_model = tf.keras.models.load_model('checkpoints/d_model_0.h5')
#g_model = tf.keras.models.load_model('checkpoints/g_model_0.h5')

In [35]:
dcgan = DCGAN(d_model, g_model, latent_dim)

In [36]:
loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
dcgan.compile(d_optimizer, g_optimizer, loss_function)

In [ ]:
dcgan.fit(normalized_ds, epochs=10, shuffle=True,
        callbacks=[custom_callback()])

In [ ]:
df_loss = [i['df_loss'] for i in training_history]
dr_loss = [i['dr_loss'] for i in training_history]
g_loss = [i['g_loss'] for i in training_history]

df_loss_ma = np.convolve(df_loss, np.ones(1000)/1000, 'valid')
dr_loss_ma = np.convolve(dr_loss, np.ones(1000)/1000, 'valid')
g_loss_ma = np.convolve(g_loss, np.ones(1000)/1000, 'valid')

In [ ]:
pyplot.figure(1, figsize = (30,20)) 
    
pyplot.subplot(221)  
pyplot.plot(df_loss_ma)  
pyplot.plot(dr_loss_ma)  
pyplot.plot(g_loss_ma)  
pyplot.title('DCGAN losses')  
pyplot.ylabel('Binary crossentropy loss')  
pyplot.xlabel('batch number')  
pyplot.legend(['discriminator fake loss', 'discriminator real loss', "generator loss"]) 

pyplot.show()